# This notebook is to import all products from your Shopify Store


In [13]:
#| default_exp products

In [14]:
#| export
import requests
import json
import os
import keyring
import sqlite3
import datetime

In [17]:
#| export
# API Configuration
api_key = keyring.get_password("Shopify", "SHOPIFY_API_KEY")
access_token = keyring.get_password("Shopify", "SHOPIFY_API_PASSWORD")
shop_url = keyring.get_password("Shopify", "SHOPIFY_SHOP_URL")
graphql_endpoint = 'https://' + shop_url + '/admin/api/2025-01/graphql.json'

# Headers for API requests
headers = {
    'X-Shopify-Access-Token': access_token,
    'Content-Type': 'application/json'
}

#| export
## Define the fields to import

In [24]:
#| export
product_fields = '''
    id
    title
    totalInventory
    tracksInventory
    updatedAt
    description
    descriptionHtml
    createdAt
    tags
    hasOutOfStockVariants
    isGiftCard
    vendor
    status
    productType
    onlineStoreUrl
    hasOnlyDefaultVariant
    mediaCount {
        count
    }
    seo {
        description
        title
    }
    variantsCount {
        count
    }
'''


In [28]:
#| export
def fetch_all_products():
    products = []
    has_next_page = True
    cursor = None
    loop_count = 0
    
    while has_next_page:
        if cursor:
            query = f'''
            {{
              products(first: 10, after: "{cursor}") {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {product_fields}
                  }}
                }}
              }}
            }}
            '''
        else:
            query = f'''
            {{
              products(first: 10) {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {product_fields}
                  }}
                }}
              }}
            }}
            '''
            
        try:
            response = requests.post(graphql_endpoint, json={'query': query}, headers=headers)
            response.raise_for_status()
            result = response.json()
            
            page_products = result['data']['products']['edges']
            for edge in page_products:
                products.append(edge['node'])
            
            has_next_page = result['data']['products']['pageInfo']['hasNextPage']
            cursor = result['data']['products']['pageInfo']['endCursor']
            
            print(f"Fetched {len(page_products)} products. Total so far: {len(products)}")
            loop_count += 1
            
        except Exception as e:
            print(f"Error fetching products: {e}")
            break
    
    return products


In [29]:
allproducts = fetch_all_products()

Fetched 10 products. Total so far: 10
Fetched 10 products. Total so far: 20
Fetched 10 products. Total so far: 30
Fetched 10 products. Total so far: 40
Fetched 10 products. Total so far: 50
Fetched 10 products. Total so far: 60
Fetched 10 products. Total so far: 70
Fetched 10 products. Total so far: 80
Fetched 10 products. Total so far: 90
Fetched 10 products. Total so far: 100
Fetched 10 products. Total so far: 110
Fetched 10 products. Total so far: 120
Fetched 10 products. Total so far: 130
Fetched 10 products. Total so far: 140
Fetched 10 products. Total so far: 150
Fetched 10 products. Total so far: 160
Fetched 10 products. Total so far: 170
Fetched 10 products. Total so far: 180
Fetched 10 products. Total so far: 190
Fetched 10 products. Total so far: 200
Fetched 10 products. Total so far: 210
Fetched 10 products. Total so far: 220
Fetched 10 products. Total so far: 230
Fetched 10 products. Total so far: 240
Fetched 10 products. Total so far: 250
Fetched 10 products. Total so far:

In [15]:
#| hide
from nbdev.showdoc import *

In [16]:
#| hide
import nbdev; nbdev.nbdev_export()